In [1]:
import requests
import csv 
from collections import Counter
from pandas import *

In [2]:
listoffiles=[]
listoffilmnames=[]
        
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False 
    
def csvgetter(user,password,*otherusernames):
    #first create a session 's'
    s=requests.session()
    #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',}
    #here we send the login data, using the session created before
    s.post("https://www.icheckmovies.com/login/", data=login_data) 
    #once again, whithin the session we get the csv link and call it 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    print('There are '+numberofusers+' users in the database: '+'\n'+'\n'.join(listoffiles))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostcheckednum=row[3]
        if is_number(mostcheckednum) is True:
            mostcheckednum=int(row[3])
        mostchecked.append(mostcheckednum)
        toplistnum=row[5]
        if is_number(mostcheckednum) is True:
            mostcheckednum=int(row[5])
        toplist.append(row[5])        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    compiled_dataframe={'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(compiled_dataframe)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

In [3]:
username1='hermesmovies'
username2='Haessar'
username3='James88'
password=''

In [4]:
csvgetter(username1,password,username2,username3)

There are 3 users in the database: 
Haessar_checklist.csv
James88_checklist.csv
hermesmovies_checklist.csv


In [5]:
user1_list=listmaker(username1)
user2_list=listmaker(username2)
user3_list=listmaker(username3)
print("user1 is "+username1+", user2 is "+username2+" and user3 is "+username3+", user#_list is a dataframe of each users list")

user1 is hermesmovies, user2 is Haessar and user3 is James88, user#_list is a dataframe of each users list


Want to make a list that both james and will have seen that I haven't seen

In [6]:
#join will and james
combo_2_3=user2_list.append(user3_list)
#return duplicates
seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 
combo_1_2_3=combo_2_3.append(user1_list)  #Create combo of all 3
seenby_1_2_3=combo_1_2_3[combo_1_2_3.duplicated('title')==True] # Show films that all 3 have seen.

In [7]:
#Attempt to find only those films not seen by all.
rec=seenby_2_3.append(seenby_1_2_3) 
nrec=rec.drop_duplicates('title')
nrec['most_checked'] = [ int(x) for x in nrec['most_checked'] ] #NEED TO CONVERT most_checked AND top_lists COLUMNS TO NUMERIC
nrec['most_checked'].convert_objects(convert_numeric=True)



/Users/gershlickdc/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


579     10350
617     56828
892     24769
935     52866
0       93917
1       20366
2       57850
3       94237
5       17276
7       31510
8       38150
10      96178
11      50433
12     112871
18      73275
...
702    83246
707    54052
709    89925
714    38450
715    56026
717    50461
721    68978
731    54862
740    80747
743    80390
745    72463
747    60868
749    50744
766    69305
792    63313
Name: most_checked, Length: 666, dtype: int64

In [8]:
nrec.sort('most_checked',ascending=False) #CURRENTLY CAN'T SORT PROPERLY DUE TO NON-NUMERIC.

,most_checked,title,top_lists
597,113657,The Dark Knight,18
12,112871,Forrest Gump,24
574,111489,The Lion King,19
562,110868,Toy Story,22
592,109492,The Lord of the Rings: The Fellowship of the Ring,23
595,108716,The Matrix,21
552,108135,Finding Nemo,15
589,104853,The Lord of the Rings: The Two Towers,19
593,104796,The Lord of the Rings: The Return of the King,21
596,103631,Inception,15


In [53]:
#this is working on a function that will do everything
def superfunction(usr1,pssword,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    password=pssword
    csvgetter(username1,password,username2,username3)
    multiname(username1,username2,username3)